In [1]:
%load_ext autoreload
%autoreload 2

import importlib

from camel.agents import ChatAgent
from camel.messages import BaseMessage


In [2]:
from src.workflow import workflow
import src.workflow.workflow
importlib.reload(src.workflow)

story = workflow.refine_user_input()

### Assessment of Story Arc

1. **Plot Hook**: 
   - The loud sound interrupting a local feast serves as a compelling event that can engage players. The panic in the village creates immediate tension and urgency, prompting characters to investigate the source of the disturbance.

2. **Environment/Setting**: 
   - The setting is somewhat implied with the mention of a village and a feast, but it lacks detailed description. To enhance immersion, consider providing more context about the village (e.g., its name, culture, notable features) and the atmosphere of the feast.

3. **Main Thread/Conflict**: 
   - The central conflict revolves around the meteorite landing and the ensuing panic. However, the story could benefit from a clearer goal or quest. For example, are the players tasked with investigating the meteorite, protecting the village, or dealing with potential threats that arise from it?

4. **NPCs**: 
   - There are no NPCs mentioned in the current description. Introducing engaging 

In [15]:
from src.agents.scene_creator import SceneCreator
from src.workflow import workflow
# importlib.reload(src.workflow)
importlib.reload(src.agents.scene_creator)

structure_agent = SceneCreator.create()
response = structure_agent.create_story_scenes(story)
print(response.msg.content)



<scenes>
    <scene>
        <title>Village Panic</title>
        <description>As the villagers of Eldergrove react to the meteorite's crash, the players witness chaos unfold. They must navigate through frightened townsfolk and help restore order while gathering information about the meteorite's location.</description>
        <encounter_type>NPC Interaction</encounter_type>
        <flavor>Quest-Giving</flavor>
        <npc_list>
            <npc>Elder Thalia, Village Elder</npc>
        </npc_list>
    </scene>
    <scene>
        <title>Investigating the Meteorite</title>
        <description>The players set out towards the meteorite's crash site, traversing the now icy landscape. They encounter ice mephits and water elementals that have emerged from the meteorite, forcing them to engage in combat.</description>
        <encounter_type>Combat</encounter_type>
        <flavor>Mixed Combat</flavor>
        <npc_list>
            <npc>Ice Mephits, Elemental Creatures</npc>
        </np

In [ ]:
#load example scenes
examples = []
with open("../res/scene-example-1.txt", "r") as file:
    examples.append(file.read())

with open("../res/scene-example-2.txt", "r") as file:
    examples.append(file.read())

with open("../res/scene-example-3.txt", "r") as file:
    examples.append(file.read())
    

In [ ]:
# setup scene writer
scene_msg = BaseMessage.make_assistant_message(
    "Scene writer",
    "You write descriptions for scenes played in dnd. "
    "You are writing for the Dungeon Master(DM), not the players. "
    "Remember to make the information easy and fast for the DM to read."
    "The scene you are writing is reference material, not a novel. The DM will be using this live at the table to run games. Split your information out into bullet points and lists, use bolding, get to the point as fast as you can, and cut anything that doesn't matter to the here-and-now (do not explain the history of each room)."
    "Here are examples of how to do it:"
    ""
    "## Example 1:"
    + examples[0] +
    "## Example 2:"
    + examples[1] +
    "## Example 3:"
    + examples[2] +
    "Don't forget you are writing descriptions for dnd scenes"
)
scene_agent = ChatAgent(scene_msg)

In [ ]:

dm_msg = BaseMessage.make_assistant_message(
    "Dungeon Master",
    "You are the dungeon master runnng the discussed game. You want to create an emerging and interesting experience for the players."
    "You need to know how the story will unfold. Which steps are optional and which one can be run in different order."
)
dm_agent = ChatAgent(dm_msg)


bm_msg = BaseMessage.make_assistant_message(
    "Beast Master",
    "You know everything about how to balance a good fight and which monsters are allowed in an dnd game."
    "Make sure the right terms are used when adressing monsters or enemies."
)
bm_agent = ChatAgent(bm_msg)

In [ ]:
from camel.tasks import Task
from camel.workforce import Workforce


def single_agent_write_scene(prompt):
    scene_response = scene_agent.step(BaseMessage.make_user_message(
        "User",
        prompt
    ))
    return scene_response.msg.content

def workforce_write_scene(prompt):
    workforce = Workforce("Dungeons and Dragons creative team"
                              ## define new_agent_kwargs
                              )
    workforce.add_single_agent_worker(
      "Agent that writes detailed descriptions about scenes",
      worker=scene_agent,
    )
    workforce.add_single_agent_worker(
      "Agent that will use the scene description to run a game of dnd. Acts as a validator for the scenes.",
      worker=dm_agent,
    )
    workforce.add_single_agent_worker(
      "Agent that knows everything about monsters and how to balance fights.",
      worker=bm_agent,
    )
    task = Task(content=prompt)
    response = workforce.process_task(task)
    return response.msg.content
# write a scene
def write_scene(current_scene, next_scene, previous_scene) -> str:
    previous_prompt = "The previous scene contains information and cues that leads to the current scene. Incoorporate these as needed and resolve them. Here is the previous scene: {{" + (previous_scene or "") + "}}. "
    next_prompt = "The scene description should contain information that leads to the next scene. Here is the next scene: {{" + (next_scene or "") + "}}. " if next_scene is not None else ""
    prompt = ("Create a scene description. "
              "Here is the prompt for the current scene you should write a description about: {{" + current_scene + "}}"
              + ( previous_prompt if previous_scene is not None else "")
              + next_prompt  +
              "Do not write a description for the next scene or the previous scene."
              "Don't forget you are writing a description for the current scene."
    )
    scene_response_content = workforce_write_scene(prompt)
    # scene_response_content = single_agent_write_scene(prompt)
    
    print(scene_response_content + "/n")
    
    return scene_response_content

cnt = 0
previous_scene = None
next_scene = None
scene_list = []
for cnt in range(0, len(all_steps) - 1):
    current_scene = all_steps[cnt]
    if cnt != len(all_steps) - 1: 
        next_scene = all_steps[cnt + 1]
        
    current_scene_desc = write_scene(current_scene, next_scene, previous_scene)
    previous_scene = current_scene_desc
    scene_list.append(current_scene_desc)
    cnt += 1
    
